In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import re 
import string
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pickle
from textblob import TextBlob
from textblob.en import Spelling 

In [2]:

# set the URL of the page to scrape
url = 'https://zenodo.org/record/3941387#.ZDRcrXZByHv'

# set the list of words to match against the file names
words_to_match_mental = ['addiction', 'adhd', 'alcoholism'
                         'anxiety','autism','bipolarreddit',
                         'bpd','depression','EDAnonymous',
                         'healthanxiety','lonely','ptsd',
                         'schizophrenia','socialanxiety','suicidewatch']
words_to_match_nonmental=['conspiracy','divorce','fitness',
                         'guns','jokes','legaladvice',
                         'meditation','parenting','personalfinance',
                         'relationships','teaching']


In [3]:
def getCSV(words):
    # send a GET request to the page
    response = requests.get(url)

    # parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find all the links that point to CSV files and contain a word from the list
    links = soup.find_all('a', class_='filename', href=re.compile(r'\.csv'))

    # download each CSV file
    dfs = []
    for link in links:
        file_url = link['href']
        file_name = re.findall(r'[^/]+\.csv', file_url)[0]
        if any(word in file_name for word in words):
            final_url='https://zenodo.org' + file_url
            response = requests.get(final_url)
            print(final_url)
            with open(file_name, 'wb') as f:
                f.write(response.content)
                print(f"Downloaded file {file_name}")
            filtered_df = pd.read_csv(file_name, usecols=['post', 'subreddit'])
            #concatenate the dataframes into a single dataframe
            dfs.append(filtered_df)
            os.remove(file_name)
    # concatenate the dataframes into a single dataframe
    combined_df = pd.concat(dfs, ignore_index=True)
    
    print("Combined CSV saved as 'combined.csv'")
    return combined_df

In [ ]:
nonMentalDF=getCSV(words_to_match_nonmental)

https://zenodo.org/record/3941387/files/conspiracy_2018_features_tfidf_256.csv?download=1
Downloaded file conspiracy_2018_features_tfidf_256.csv
https://zenodo.org/record/3941387/files/conspiracy_2019_features_tfidf_256.csv?download=1
Downloaded file conspiracy_2019_features_tfidf_256.csv
https://zenodo.org/record/3941387/files/conspiracy_post_features_tfidf_256.csv?download=1
Downloaded file conspiracy_post_features_tfidf_256.csv
https://zenodo.org/record/3941387/files/conspiracy_pre_features_tfidf_256.csv?download=1
Downloaded file conspiracy_pre_features_tfidf_256.csv
https://zenodo.org/record/3941387/files/divorce_2018_features_tfidf_256.csv?download=1
Downloaded file divorce_2018_features_tfidf_256.csv
https://zenodo.org/record/3941387/files/divorce_2019_features_tfidf_256.csv?download=1
Downloaded file divorce_2019_features_tfidf_256.csv
https://zenodo.org/record/3941387/files/divorce_post_features_tfidf_256.csv?download=1
Downloaded file divorce_post_features_tfidf_256.csv
https

In [ ]:
nonMentalDF.head()

In [ ]:
nonMentalDF['subreddit'].value_counts()

In [ ]:
MentalDF['subreddit'].value_counts()

In [ ]:
print(sum(nonMentalDF['subreddit'].value_counts()))
print(sum(MentalDF['subreddit'].value_counts()))


In [ ]:
MentalDF.to_csv('combined_positive_mental_health.csv', index=False)
nonMentalDF.to_csv('combined_negative_mental_health.csv', index=False)


In [ ]:
MentalDF=pd.read_csv('combined_positive_mental_health.csv')
nonMentalDF=pd.read_csv('combined_negative_mental_health.csv')

In [ ]:
from textblob.en import Spelling        
import re

textToLower = ""

with open("train.txt","r") as f1:           # Open our source file
    text = f1.read()                                  # Read the file                 
    textToLower = text.lower()                        # Lower all the capital letters

words = re.findall("[a-z]+", textToLower)             # Find all the words and place them into a list    
oneString = " ".join(words)                           # Join them into one string

pathToFile = "train.txt"                              # The path we want to store our stats file at
spelling = Spelling(path = pathToFile)                # Connect the path to the Spelling object
spelling.train(oneString, pathToFile)

In [ ]:
oneString

In [ ]:
def convert_lowercase(text):
    text = text.lower()
    return text
def remove_url(text):
    re_url = re.compile('https?://\S+|www\.\S+')
    return re_url.sub('', text)
exclude = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))
def remove_stopwords(text):
    new_list = []
    words = word_tokenize(text)
    stopwrds = stopwords.words('english')
    for word in words:
        if word not in stopwrds:
            new_list.append(word)
    return ' '.join(new_list)
def steaming(text):
    ps = PorterStemmer()
    tokens = text.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)


In [ ]:
def preprocessed(full_data):
    full_data=full_data.dropna(how='any')
    full_data['subreddit'].value_counts().reset_index(name='count')
    full_data['post'] = full_data['post'].apply(convert_lowercase)
    full_data['post'] = full_data['post'].apply(remove_url)
    full_data['post'] = full_data['post'].apply(remove_punc)
    full_data['post'] = full_data['post'].apply(remove_stopwords)
    full_data['post'] = full_data['post'].apply(lemmatize_words)
    full_data['post']=full_data['post'].astype(str).apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
    full_data['post'] = full_data['post'].apply(correct_spelling)
    return full_data

In [ ]:
MentalDFProcessed=preprocessed(MentalDF)

In [ ]:
nonMentalDFProcessed=preprocessed(nonMentalDF)

In [ ]:
nonMentalDFProcessed=nonMentalDFProcessed.drop(['subreddit'],axis=1)
nonMentalDFProcessed=['subreddit']='nonmental'

In [ ]:
MentalDFProcessed

In [ ]:
MentalDFProcessed.to_csv('../datasets/positive_reddit_combined.csv')
nonMentalDFProcessed.to_csv('../datasets/negative_reddit_combined.csv')

In [ ]:
nonMentalDFProcessed